## 연습 문제 2. 영화 추천하기

데이터 셋 출처: https://www.kaggle.com/datasets/hijest/genre-classification-dataset-imdb

IMDB는 영화 리뷰를 제공하는 웹 서비스입니다. 이 사이트에서 크롤링한 영화 제목, 장르, 줄거리 데이터 셋을 가지고 간단한 추천을 구현해보고자 합니다. 사용자는 자연어 형태로 본인이 원하는 영화의 장르, 특징, 줄거리 등을 입력하면 tf-idf를 이용하여 적절한 영화를 추천해주는 시스템을 개발해보세요.

In [94]:
import pandas as pd
from tqdm import tqdm

tqdm.pandas()
df = pd.read_csv("./data/movies_preprocessed.csv", encoding="utf-8")


In [95]:
df["content_tokens"] = df["content_tokens"].progress_apply(lambda x: eval(x))

100%|██████████████████████████████████████████████████████████████████████████| 54214/54214 [00:07<00:00, 7520.18it/s]


In [97]:
df

,title,genre,content,content_tokens
0,Oscar et la dame rose (2009),drama,Listening in to a conversation between his doc...,"[listen, convers, hi, doctor, parent, ,, 10, -..."
1,Cupid (1997),thriller,A brother and sister with a past incestuous re...,"[brother, sister, past, incestu, relationship,..."
2,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fiel...,"[bu, empti, student, field, trip, museum, natu..."
3,The Secret Sin (1915),drama,To help their unemployed father make ends meet...,"[help, unemploy, father, make, end, meet, ,, e..."
4,The Unrecovered (2007),drama,The film's title refers not only to the un-rec...,"[film, ', titl, refer, onli, un, -, recov, bod..."
...,...,...,...,...
54209,"""Bonino"" (1953)",comedy,This short-lived NBC live sitcom centered on B...,"[thi, short, -, live, nbc, live, sitcom, cente..."
54210,Dead Girls Don't Cry (????),horror,The NEXT Generation of EXPLOITATION. The siste...,"[next, gener, exploit, ., sister, kapa, -, bay..."
54211,Ronald Goedemondt: Ze bestaan echt (2008),documentary,"Ze bestaan echt, is a stand-up comedy about gr...","[ze, bestaan, echt, ,, stand, -, comedi, grow,..."
54212,Make Your Own Bed (1944),comedy,Walter and Vivian live in the country and have...,"[walter, vivian, live, countri, difficult, tim..."


In [100]:
df[df["title"]=="Oscar et la dame rose (2009)"]["content"].values

array(['Listening in to a conversation between his doctor and parents, 10-year-old Oscar learns what nobody has the courage to tell him. He only has a few weeks to live. Furious, he refuses to speak to anyone except straight-talking Rose, the lady in pink he meets on the hospital stairs. As Christmas approaches, Rose uses her fantastical experiences as a professional wrestler, her imagination, wit and charm to allow Oscar to live life and love to the full, in the company of his friends Pop Corn, Einstein, Bacon and childhood sweetheart Peggy Blue.\n'],
      dtype=object)

In [96]:
for i in range(len(df["title"])):
    df["content_tokens"][i].append(df["genre"][i])

In [99]:
contains_drama = any(df['content_tokens'].str.contains('documentary', case=False))
print("Does 'content_tokens' column contain 'drama'? :", contains_drama)

Does 'combined' column contain 'drama'? : True


In [131]:
df[df["title"]=="Oscar et la dame rose (2009)"]["content_tokens"].values

array([list(['listen', 'convers', 'hi', 'doctor', 'parent', ',', '10', '-', 'year', '-', 'old', 'oscar', 'learn', 'nobodi', 'ha', 'courag', 'tell', '.', 'onli', 'ha', 'week', 'live', '.', 'furiou', ',', 'refus', 'speak', 'anyon', 'except', 'straight', '-', 'talk', 'rose', ',', 'ladi', 'pink', 'meet', 'hospit', 'stair', '.', 'christma', 'approach', ',', 'rose', 'use', 'fantast', 'experi', 'profession', 'wrestler', ',', 'imagin', ',', 'wit', 'charm', 'allow', 'oscar', 'live', 'life', 'love', 'full', ',', 'compani', 'hi', 'friend', 'pop', 'corn', ',', 'einstein', ',', 'bacon', 'childhood', 'sweetheart', 'peggi', 'blue', '.', 'drama'])],
      dtype=object)

In [101]:
df["genre"].values

array(['drama', 'thriller', 'adult', ..., 'documentary', 'comedy',
       'history'], dtype=object)

In [102]:
from sklearn.feature_extraction.text import TfidfVectorizer

def dummy_func(doc):
    return doc

tfidf = TfidfVectorizer(
    analyzer = "word",
    tokenizer = dummy_func,
    preprocessor = dummy_func,
    token_pattern = None
)

In [103]:
tfidf_csr_matrix = tfidf.fit_transform(df["content_tokens"])

In [104]:
tfidf_csr_matrix

<54214x93627 sparse matrix of type '<class 'numpy.float64'>'
	with 2987484 stored elements in Compressed Sparse Row format>

In [105]:
# tfidf_vector_list = tfidf_csr_matrix.toarray() --> 메모리 에러

In [106]:
#tfidf.vocabulary_

In [107]:
len(tfidf.vocabulary_)

93627

In [108]:
print(tfidf_csr_matrix[0])

  (0, 24119)	0.05175856311111811
  (0, 11185)	0.12472681184294948
  (0, 62609)	0.18108102787307076
  (0, 80567)	0.14352347404261556
  (0, 16371)	0.10871024963702527
  (0, 7820)	0.19115511583187178
  (0, 25598)	0.1980925112304484
  (0, 19035)	0.19032038518280114
  (0, 64717)	0.1251099968516713
  (0, 30738)	0.06725520403031778
  (0, 18124)	0.10152401240010492
  (0, 31021)	0.10384449438334185
  (0, 49320)	0.06417381770660821
  (0, 48340)	0.05397078855099042
  (0, 4122)	0.11106105572456552
  (0, 15906)	0.1251529856640617
  (0, 90838)	0.10463001981298152
  (0, 39196)	0.11296074089926245
  (0, 91255)	0.17123175591814827
  (0, 65734)	0.11206181842310099
  (0, 27650)	0.08720659963078775
  (0, 28207)	0.1522459622732843
  (0, 87017)	0.08059362871090457
  (0, 5696)	0.11627124413880753
  (0, 16722)	0.13026121174724356
  :	:
  (0, 27537)	0.12671278748924783
  (0, 5486)	0.11886554130543489
  (0, 77848)	0.11601355885110883
  (0, 68496)	0.11133581375491972
  (0, 31113)	0.1584628465013306
  (0, 48776)	

In [109]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

In [110]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [111]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_word_list = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\82108\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [112]:
def preprocess(text):
    text = text.lower()
    tokens = tokenizer.tokenize(text)
    stems = [stemmer.stem(token) for token in tokens]
    filtered_tokens = [x for x in stems if x not in stop_word_list]
    return filtered_tokens

In [122]:
query = input("movie feature: ")
nouns = preprocess(query)

movie feature:  brother and sister murderous relationship murders woman thriller


In [123]:
nouns

['brother', 'sister', 'murder', 'relationship', 'murder', 'woman', 'thriller']

In [124]:
query_csr_matrix = tfidf.transform([nouns])

In [125]:
vocab = tfidf.vocabulary_

In [126]:
for noun in nouns:
    try:
        print(noun, vocab[noun])
    except:
        continue

brother 12866
sister 76138
murder 56353
relationship 68688
murder 56353
woman 91006
thriller 82963


In [127]:
print(query_csr_matrix)

  (0, 91006)	0.28843308008197244
  (0, 82963)	0.3624561490301186
  (0, 76138)	0.35409671276760385
  (0, 68688)	0.3253284514439356
  (0, 56353)	0.6674768972067466
  (0, 12866)	0.329671665036905


In [128]:
from sklearn.metrics.pairwise import cosine_similarity

In [129]:
def search(query):
    query_nouns = preprocess(query)
    query_tfidf = tfidf.transform([query_nouns])
    similarities = cosine_similarity(query_tfidf, tfidf_csr_matrix).flatten()
    top_similarities = sorted(similarities, reverse=True)[:10]        # similarities 높은거 5개만
    top_indices = similarities.argsort()[-10:][::-1]                  # top5의 인덱스 넘버 추출
    top_titles = [df.iloc[i]["title"] for i in top_indices]          # top5의 기사 제목 추출
    for similarity, title in zip(top_similarities, top_titles):
        print(similarity, title)

In [130]:
search(query)

0.6678861137803139 Cupid (1997)
0.34518350167105094 A Yellow Streak (1927)
0.3288153667578964 Hotel Chelsea (2009)
0.3030492532319431 Water (2004/I)
0.3002802813263131 Murder Book (2015)
0.28604984911889014 Franco e Ciccio ...ladro e guardia (1969)
0.284315874880685 Broken Reality (1994)
0.2775667964181764 Nurse shimai: Momoiro shinsatsu-shitsu (2003)
0.27656912414771195 "Run of the House" (2003)
0.275578630376161 The Initiate (1998)
